In [1]:
import requests
from PIL import Image
from pprint import pprint
import json
import pandas as pd

In [38]:
api_key = "vytvIDyAwxQXP87gn0Defwiltz4391uInF8bObyE"
url_weather = "https://api.nasa.gov/insight_weather/?"
url_images = "https://api.nasa.gov/mars-photos/api/v1/rovers/{}/photos?"
rovers = ["curiosity"]#, "opportunity", "spirit"]
# All cameras with the exception of MARDI - The Mars Descent Imager (as there are no photos for any day aside from the descent date)
curiosity_cameras = ["FHAZ", "RHAZ", "MAST",
                     "CHEMCAM","MAHLI", "NAVCAM"]
# [url_images.format(x) for x in rovers]
# API Mars Weather Docs: https://api.nasa.gov/assets/insight/InSight%20Weather%20API%20Documentation.pdf

In [29]:
payload_weather = {
    "api_key": api_key,
    "feedtype": "json"
}
payload_images = {
    "api_key": "DEMO_KEY"#api_key
}

In [4]:
# resp = requests.get(url, params=payload)
resp = requests.get(url_weather, params=payload_weather)

In [76]:
def get_weather(url_weather, payload):
    resp = requests.get(url_weather, params=payload)
    return json.loads(resp.content)

def get_images(url_images, rover, day, payload):
    print("Getting the images for {} rover on Day {}.".format(
    rover, day[:10]
    ))
    payload_sol = payload
    payload_sol["earth_date"] = day[:10]
    rover_url = url_images.format(rover)
    resp = requests.get(rover_url, params=payload_sol)
    return json.loads(resp.content)

def get_images_on_day_list(images_json):
    photos = images_json["photos"]
    first_images = []
    for camera in curiosity_cameras:
        first_image = "None Found"
        for photo in photos:
            if photo["camera"]["name"] == camera:
                first_image = photo["img_src"]
                break
        first_images.append({
            camera: first_image
        })
    return first_images

def combine_to_dict(day_images):
    full_dict = {}
    for camera in curiosity_cameras:
        full_dict[camera] = []
    for day in day_images:
        for camera in day:
            cam = list(camera.keys())[0]
            full_dict[cam].append(camera[cam])
    return full_dict

def get_images_df(url, days, rover, payload):
    day_images = []
    for day in days:
        day_images.append(
            get_images_on_day_list(
                get_images(url, rover, day, payload)))
    return pd.DataFrame(combine_to_dict(day_images))

In [6]:
# resp.content

In [7]:
cont = json.loads(resp.content)
cont2 = {
    key: cont[key] for key in list(cont.keys())[:-2]
}
# pprint(cont2)
cont_pd = pd.DataFrame(cont2)

In [8]:
weather = []
for key,val in cont2.items():
    pre = val["PRE"]
    del val["PRE"]
    del val["WD"]
    val = {**val, **pre}
    updated = {**{"Sol":key}, **val}
    weather.append(updated)
# weather

In [9]:
weather_pd = pd.DataFrame(weather)
new_order = ["Sol"] + [x for x in weather_pd.columns.tolist() if x != "Sol"]
weather_pd = weather_pd[new_order]

In [10]:
weather_pd

,Sol,First_UTC,Last_UTC,Month_ordinal,Northern_season,Season,Southern_season,av,ct,mn,mx
0,809,2021-03-06T10:57:03Z,2021-03-07T11:36:33Z,12,late winter,winter,late summer,727.644,103645,706.7370,744.8863
1,810,2021-03-07T11:36:38Z,2021-03-08T12:16:13Z,12,late winter,winter,late summer,728.062,104975,707.2886,744.3631
2,811,2021-03-08T12:16:13Z,2021-03-09T12:55:48Z,12,late winter,winter,late summer,728.219,149673,707.2492,745.7073
3,812,2021-03-09T12:55:49Z,2021-03-10T13:35:24Z,12,late winter,winter,late summer,727.699,105992,706.3650,742.9495
4,813,2021-03-10T13:35:28Z,2021-03-11T14:14:59Z,12,late winter,winter,late summer,727.909,100620,706.2391,743.8608
5,814,2021-03-11T14:15:03Z,2021-03-12T14:54:34Z,12,late winter,winter,late summer,728.121,104121,706.9009,745.4649
6,815,2021-03-12T14:54:38Z,2021-03-13T15:34:09Z,12,late winter,winter,late summer,728.378,153082,708.4211,744.9279


In [11]:
days = weather_pd.First_UTC

In [12]:
days

0    2021-03-06T10:57:03Z
1    2021-03-07T11:36:38Z
2    2021-03-08T12:16:13Z
3    2021-03-09T12:55:49Z
4    2021-03-10T13:35:28Z
5    2021-03-11T14:15:03Z
6    2021-03-12T14:54:38Z
Name: First_UTC, dtype: object

In [31]:
images_json = get_images(url_images,rovers[0],days[0],payload_images)

Getting the images for curiosity rover on Day 2021-03-06T10:57:03Z.


In [32]:
images_json

{'photos': [{'id': 809358,
   'sol': 3051,
   'camera': {'id': 20,
    'name': 'FHAZ',
    'rover_id': 5,
    'full_name': 'Front Hazard Avoidance Camera'},
   'img_src': 'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03051/opgs/edr/fcam/FLB_668364089EDR_F0870696FHAZ00337M_.JPG',
   'earth_date': '2021-03-06',
   'rover': {'id': 5,
    'name': 'Curiosity',
    'landing_date': '2012-08-06',
    'launch_date': '2011-11-26',
    'status': 'active'}},
  {'id': 809359,
   'sol': 3051,
   'camera': {'id': 20,
    'name': 'FHAZ',
    'rover_id': 5,
    'full_name': 'Front Hazard Avoidance Camera'},
   'img_src': 'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03051/opgs/edr/fcam/FRB_668364089EDR_F0870696FHAZ00337M_.JPG',
   'earth_date': '2021-03-06',
   'rover': {'id': 5,
    'name': 'Curiosity',
    'landing_date': '2012-08-06',
    'launch_date': '2011-11-26',
    'status': 'active'}},
  {'id': 809360,
   'sol': 3051,
   'camera': {'id': 20,
   

In [33]:
pd.DataFrame(images_json["photos"])

,id,sol,camera,img_src,earth_date,rover
0,809358,3051,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
1,809359,3051,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
2,809360,3051,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
3,809361,3051,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
4,809362,3051,"{'id': 20, 'name': 'FHAZ', 'rover_id': 5, 'ful...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
...,...,...,...,...,...,...
451,809373,3051,"{'id': 26, 'name': 'NAVCAM', 'rover_id': 5, 'f...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
452,809374,3051,"{'id': 26, 'name': 'NAVCAM', 'rover_id': 5, 'f...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
453,809375,3051,"{'id': 26, 'name': 'NAVCAM', 'rover_id': 5, 'f...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."
454,809376,3051,"{'id': 26, 'name': 'NAVCAM', 'rover_id': 5, 'f...",https://mars.nasa.gov/msl-raw-images/proj/msl/...,2021-03-06,"{'id': 5, 'name': 'Curiosity', 'landing_date':..."


In [58]:
one_day = get_images_on_day_list(images_json)

In [65]:
pd.DataFrame(combine_to_dict([one_day]))

,FHAZ,RHAZ,MAST,CHEMCAM,MAHLI,NAVCAM
0,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...


In [74]:
images_df = get_images_df(url_images,days,rovers[0],payload_images)

Getting the images for curiosity rover on Day 2021-03-06T10:57:03Z.
Getting the images for curiosity rover on Day 2021-03-07T11:36:38Z.
Getting the images for curiosity rover on Day 2021-03-08T12:16:13Z.
Getting the images for curiosity rover on Day 2021-03-09T12:55:49Z.
Getting the images for curiosity rover on Day 2021-03-10T13:35:28Z.
Getting the images for curiosity rover on Day 2021-03-11T14:15:03Z.
Getting the images for curiosity rover on Day 2021-03-12T14:54:38Z.


In [77]:
images_df

,FHAZ,RHAZ,MAST,CHEMCAM,MAHLI,NAVCAM
0,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...
1,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,None Found,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...
2,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,None Found,None Found,None Found,None Found
3,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,None Found,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...
4,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,None Found,https://mars.nasa.gov/msl-raw-images/proj/msl/...
5,None Found,None Found,None Found,None Found,None Found,None Found
6,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,None Found,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...


In [81]:
full_df = weather_pd.join(images_df)

In [82]:
full_df

,Sol,First_UTC,Last_UTC,Month_ordinal,Northern_season,Season,Southern_season,av,ct,mn,mx,FHAZ,RHAZ,MAST,CHEMCAM,MAHLI,NAVCAM
0,809,2021-03-06T10:57:03Z,2021-03-07T11:36:33Z,12,late winter,winter,late summer,727.644,103645,706.7370,744.8863,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...
1,810,2021-03-07T11:36:38Z,2021-03-08T12:16:13Z,12,late winter,winter,late summer,728.062,104975,707.2886,744.3631,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,None Found,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...
2,811,2021-03-08T12:16:13Z,2021-03-09T12:55:48Z,12,late winter,winter,late summer,728.219,149673,707.2492,745.7073,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,None Found,None Found,None Found,None Found
3,812,2021-03-09T12:55:49Z,2021-03-10T13:35:24Z,12,late winter,winter,late summer,727.699,105992,706.3650,742.9495,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,None Found,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...
4,813,2021-03-10T13:35:28Z,2021-03-11T14:14:59Z,12,late winter,winter,late summer,727.909,100620,706.2391,743.8608,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,None Found,https://mars.nasa.gov/msl-raw-images/proj/msl/...
5,814,2021-03-11T14:15:03Z,2021-03-12T14:54:34Z,12,late winter,winter,late summer,728.121,104121,706.9009,745.4649,None Found,None Found,None Found,None Found,None Found,None Found
6,815,2021-03-12T14:54:38Z,2021-03-13T15:34:09Z,12,late winter,winter,late summer,728.378,153082,708.4211,744.9279,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/proj/msl/...,https://mars.nasa.gov/msl-raw-images/msss/0305...,None Found,https://mars.nasa.gov/msl-raw-images/msss/0305...,https://mars.nasa.gov/msl-raw-images/proj/msl/...


In [17]:
# exmp = cont_pd.iloc[0]
# download_url = "https://epic.gsfc.nasa.gov/epic-archive/jpg/{}.jpg".format(exmp["image"])

In [18]:
# im = Image.open(requests.get(download_url, stream=True).raw)

In [19]:
# download_url

In [20]:
# im

SyntaxError: invalid syntax (<ipython-input-42-c1cc428ec200>, line 1)